In [2]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException, ElementClickInterceptedException
from selenium.webdriver.common.by import By
import pandas as pd

In [121]:
# Declare browser
s = Service('C://Users//hantt3//Downloads//chromedriver_win32')
driver = webdriver.Chrome(service=s)
url = 'https://www.youtube.com/watch?v=uKB7FK2ZLRc'

In [122]:

def scrape(url):

    driver.get(url)
    driver.maximize_window()
    sleep(5)
    
    name_comment, text_comment = [], []
    try:
        # Extract the elements storing the video title and
        # comment section.
        title = driver.find_element(By.XPATH, '//*[@id="title"]/h1/yt-formatted-string').text
        comment_section = driver.find_element(By.XPATH, '//*[@id="comments"]')
    except NoSuchElementException:

        print("NoSuchElementException")

    # Scroll into view the comment section, then allow some time
    # for everything to be loaded as necessary.
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(7)

    # Scroll all the way down to the bottom in order to get all the
    # elements loaded (since Youtube dynamically loads them).
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # Scroll down 'til "next load".
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load everything thus far.
        sleep(2)

        # Calculate new scroll height and compare with last scroll height.
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # One last scroll just in case.
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

    try:
        # Extract the elements storing the usernames and comments.
        username_elems = driver.find_elements(By.XPATH, '//*[@id="author-text"]')
        name_comment = [elem.text for elem in username_elems] + name_comment
        comment_elems = driver.find_elements(By.XPATH, '//*[@id="content-text"]')
        text_comment = [elem.text for elem in comment_elems] + text_comment
    except NoSuchElementException:
        print('NoSuchElementException')

    print("> VIDEO TITLE: " + title + "\n")

    df = pd.DataFrame(list(zip(name_comment, text_comment)), 
                   columns = ['name_comment', 'content_comment'])

    return df
    driver.close()

In [123]:
data = scrape(url)

> VIDEO TITLE: Về già tưởng có tiền vào viện dưỡng lão không phiền con cháu để rồi vỡ mộng - Chân Thiện Mỹ



In [125]:
data.shape

(125, 2)

In [126]:
#convert to csv file
data.to_csv("commentYoutube.csv", index = False)